主题是神经网络的学习，“学习”是指从训练数据中自动获取最优权重参数的过程。

我们会导入**损失函数**这一指标，找出尽可能小的损失函数的值


## 4.1 从数据中学习

“从数据中学习”，是指可以由数据自动决定权重参数的值。

### 4.1.1　数据驱动

在计算机视觉领域，常用的特征量包括SIFT、SURF和HOG，使用这些特征量将图像数据转换为向量，然后对转换后的向量使用机器学习中的SVM、KNN等分类器进行学习

与待处理的问题无关，神经网络可以将数据直接作为原始数据，进行“端对端”的学习

### 4.1.2　训练数据和测试数据

使用训练数据进行学习寻找最优的参数；使用测试数据评价训练得到的模型的实际能力。

为什么要分为训练数据和测试数据？**因为追求的是模型的泛化能力**

只对某个数据集过度拟合的状态称为**过拟合**

## 4.2 损失函数

### 4.2.1　均方误差

$$
E = \frac{1}{2} \sum_{k} (y_{k} - t_{k})^2
$$
这里，$y_k$是表示神经网络的输出，$t_k$表示监督数据(训练数据)，k表示数据的维数

In [1]:
#来实现这个均方误差
import numpy as np
def mean_squared_error(y, t):
    return 0.5*np.sum((y-t)**2)
#设2为正确解
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
#例子1：此时“2”的概率最高为0.6
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(mean_squared_error(np.array(y), np.array(t)))
#例子2：此时“7”的概率最高为0.6
yy = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(mean_squared_error(np.array(yy), np.array(t)))

0.09750000000000003
0.5975


**我们会发现，第一个例子的损失函数的值更小，和监督数据之间的误差较小，表明第一个例子的输出结果与监督数据更加吻合**

### 4.2.2　交叉熵误差

$$
E = - \sum_{k} t_{k} \log y_{k}
$$

In [2]:
#实现交叉熵误差
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))
#为什么要加上delta？因为当出现log0时，会变为无限大会导致后续计算无法进行。

In [3]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

y2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(cross_entropy_error(np.array(y2), np.array(t)))

0.510825457099338
2.302584092994546


### 4.2.3　mini-batch学习

前两个方法考虑的都是针对单个数据的损失函数，如果要求所有训练数据的损失函数的总和，以交叉熵误差为例，可以写成下面的式

$$
E = - \frac{1}{N} \sum_{n} \sum_{k} t_{nk} \log y_{nk}
$$

数据由$N$个，$t_{nk}$表示第n个数据的第k个元素的值

从60000个训练数据中随机选择100笔，再用这100笔数据进行学习。这种学习方式称为**mini-batch学习**

**在这之前，先来看一下用于读入MNIST数据集的代码**

In [4]:
import sys, os
import numpy as np
from dataset.mnist import load_mnist
sys.path.append(os.pardir)
#通过设定参数one_hot_label=True，可以得到one-hot表示（即仅正确解标签为1，其余为0的数据结构）
(x_train, t_train), (x_test, t_test) =load_mnist(normalize=True, one_hot_label=True)
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000, 10)

(60000, 784)
(60000, 10)


**如何从训练数据随机抽取110笔数据？使用NumPy的np.random.choice()**

In [5]:
#从 x_train 的维度中获取训练集的总大小（即 60000），并存入 train_size 变量
train_size=x_train.shape[0]
#设置小批量（mini-batch）的大小,在每一步训练中随机抽取 10 个数据点
batch_size=10
#从 0 到 train_size - 1（即 0 到 59999）的整数中,随机抽取 batch_size（即 10）个数字。
batch_mask = np.random.choice(train_size, batch_size)#batch_mask是NumPy数组

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [6]:
np.random.choice(60000, 10)

array([45169, 39081, 48616, 39566, 29487, 50867,  1338, 44510,   646,
       12157], dtype=int32)

### 4.2.4　mini-batch版交叉熵误差的实现

**来实现一个可以同时处理单个数据和批量数据（数据作为batch集中输入）两种情况的函数**

8999

爱给网